<a href="https://colab.research.google.com/github/AnaGarciaDelAlamo/clasificador-ingl-s/blob/main/Clasificador_ingl%C3%A9s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#instalamos el conjunto de datos de entrenamiento
!pip install datasets
#libreria con todods los modelos
!pip install transformers
#nos permite entrenar
!pip install accelerate -U
#nos permite evaluar el modelo
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00


In [ ]:
#Importamos el dataset
from datasets import load_dataset

raw_datasets = load_dataset("tyqiangz/multilingual-sentiments", "english")
raw_datasets

README.md:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

multilingual-sentiments.py:   0%|          | 0.00/6.23k [00:00<?, ?B/s]

english/train/0000.parquet:   0%|          | 0.00/156k [00:00<?, ?B/s]

english/validation/0000.parquet:   0%|          | 0.00/29.6k [00:00<?, ?B/s]

english/test/0000.parquet:   0%|          | 0.00/65.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1839 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/324 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/870 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 1839
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 324
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 870
    })
})

In [ ]:
#cargamos los datos de entrenamiento en una variable y mostramos el primero
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'text': 'okay i\\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\\u2019mon America aren\\u2019t you sick of her yet? (sorry) ',
 'source': 'sem_eval_2017',
 'label': 2}

In [ ]:
#Características del dataset
raw_train_dataset.features

{'text': Value(dtype='string', id=None),
 'source': Value(dtype='string', id=None),
 'label': ClassLabel(names=['positive', 'neutral', 'negative'], id=None)}

In [ ]:
#tokenizamos y creamos las secuencias del dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoConfig

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
config = AutoConfig.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
#tokenizamos los textos y agruparlos en batch para poder pasarlos al modelo
def tokenizer_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenizer_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/1839 [00:00<?, ? examples/s]

Map:   0%|          | 0/324 [00:00<?, ? examples/s]

Map:   0%|          | 0/870 [00:00<?, ? examples/s]

In [ ]:
#Importamos los parámetros de entrenamiento, vamos a importar unos paráemtros aleatorios porque es dificil
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
#Cargamos el modelo de clasificación de secuencias e indicamos el que tenemos y que tenga en cuenta 3 posibles etiquetas
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#cargamos el método para poder computar el desempeñlo del modelo, compara las etiquetas y las predicciones
import evaluate
import numpy as np

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
#definimos los argumentos del entrenamiento que son los parametros de antes y lo vamos a evaluar cada epoc
training_args = TrainingArguments(
    "test-trainer",
    evaluation_strategy="epoch",  # Evalúa al final de cada epoch
    logging_steps=10,  # Log cada 10 pasos (se mostrará en la tabla)
    log_level="info",  # Asegura que el nivel de log sea adecuado
    report_to="none",  # No reportar a archivos ni servicios externos
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#vamos a indicar como va a ser el entrenamieto, modelo, argumentos, datos a entrenar, datos para evaluar, como agrupamos
#los datos, tokenizador a usar y la política para medir las métricas
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
import wandb
wandb.init(mode="disabled")

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: source, text. If source, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1,839
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 690
  Number of trainable parameters = 109,484,547


Epoch,Training Loss,Validation Loss,Accuracy
1,0.848000,0.698036,0.682099
2,0.526100,0.719981,0.722222
3,0.093800,1.106423,0.706790


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: source, text. If source, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 324
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: source, text. If source, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 324
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/model.safetensors
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/ch

TrainOutput(global_step=690, training_loss=0.5719327374644901, metrics={'train_runtime': 87.7924, 'train_samples_per_second': 62.841, 'train_steps_per_second': 7.859, 'total_flos': 120220220592360.0, 'train_loss': 0.5719327374644901, 'epoch': 3.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Datos Nuevas Tecnologías/Clasificador2")

Saving model checkpoint to /content/drive/MyDrive/Datos Nuevas Tecnologías/Clasificador2
Configuration saved in /content/drive/MyDrive/Datos Nuevas Tecnologías/Clasificador2/config.json
Model weights saved in /content/drive/MyDrive/Datos Nuevas Tecnologías/Clasificador2/model.safetensors
tokenizer config file saved in /content/drive/MyDrive/Datos Nuevas Tecnologías/Clasificador2/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Datos Nuevas Tecnologías/Clasificador2/special_tokens_map.json


In [ ]:
from transformers import pipeline
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoConfig


#ponemos un modelo de encoder que clasifique las secuencias en inglés
checkpoint = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google-bert--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "_name_or_path": "google-bert/bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.44.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--google-bert--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734

In [ ]:
new_model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Datos Nuevas Tecnologías/Clasificador2")

clf = pipeline("text-classification", model=new_model, tokenizer=tokenizer)

loading configuration file /content/drive/MyDrive/Datos Nuevas Tecnologías/Clasificador2/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.44.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 3

In [ ]:
validation_data = raw_datasets["validation"]

In [ ]:
validation_data[178]

{'text': 'Did I mention I am going to the Blue Jays game on Friday? I am going to the Blue Jays game on Friday. David Price is pitching. ',
 'source': 'sem_eval_2017',
 'label': 1}

In [ ]:
clf(validation_data[178]["text"])

[{'label': 'LABEL_1', 'score': 0.9833954572677612}]

In [ ]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 1839
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 324
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 870
    })
})


In [ ]:
#evaluar el conjunto de test
trainer.evaluate(tokenized_datasets["test"])

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: source, text. If source, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 870
  Batch size = 8


{'eval_loss': 1.2370972633361816,
 'eval_accuracy': 0.674712643678161,
 'eval_runtime': 2.5545,
 'eval_samples_per_second': 340.576,
 'eval_steps_per_second': 42.67,
 'epoch': 3.0}